In [12]:
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import torch
import torch.nn as nn
from utils import *
from dataloaders.RealBlur import *
import matplotlib.pyplot as plt
from torchvision import transforms
import einops
from dataloaders.ColorizationDataset import ColorizationDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
dataset = ColorizationDataset(train=True, width=224, height=224, dataset_root ="/Users/sagarasanghavi/Desktop/Photos")
print(dataset.filenames)

['Gradphoto', 'IMG_1042', 'IMG_1081', 'IMG_1252', 'IMG_4493', 'Photo']


In [18]:
# sanity check
def show_image(img):
    plt.figure()
    plt.imshow(cv2.cvtColor(img.transpose((1, 2, 0)), cv2.COLOR_BGR2RGB))
    plt.show()

_, gt = dataset[1]
show_image(gt)

Image title /Users/sagarasanghavi/Desktop/Photos/JPEGImages/IMG_1042.jpg


[ WARN:0@3067.565] global loadsave.cpp:244 findDecoder imread_('/Users/sagarasanghavi/Desktop/Photos/JPEGImages/IMG_1042.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /Users/runner/miniforge3/conda-bld/libopencv_1678824230546/work/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
from models.axialtransformer import *

blur, gt = dataset[0]

cropped_image = gt[None, :, :512, :512]
cropped_blur = blur[None, :, :512, :512]

cropped_image_torch = torch.from_numpy(cropped_image).float()
print(cropped_image_torch.shape)

conv1x1 = nn.Conv2d(3, 128, 1)
transformer = AxialImageTransformer(dim=128, depth=12, reversible=True)

out = transformer(conv1x1(cropped_image_torch))
print(out.shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 128, 224, 224])


In [17]:
from models.unet import *

unet_model = UNet(n_channels=3, n_classes=20, bilinear=False)
unet_model()